### Recursos


- https://public.tableau.com/app/learn/sample-data
- https://github.com/plotly/datasets/tree/master
- https://github.com/vega/vega-datasets/tree/main/data


### Bibliotecas


In [ ]:
import os
import pathlib
import traceback
import sys
import duckdb
import pandas as pd
import plotly.express as px
from urllib.request import urlretrieve


# Display all columns
pd.set_option('display.max_columns', None)
# Display all characters inside a column cell
pd.set_option("display.max_colwidth", None)
# Display all rows
pd.set_option("display.max_rows", None)
# Display all intems in a nested list inside a column cell
pd.set_option("display.max_seq_item", None)

# Enable create sql cells
%load_ext sql

In [ ]:
# Automatically convert sql query result to pandas dataframe
%config SqlMagic.autopandas=True
# Disable message feedback printend after executing SQL command
%config SqlMagic.feedback=False
# Disable connection string display, connection string may contain sensitive
# information
%config SqlMagic.displaycon=False
# Enable named parameters
%config SqlMagic.named_parameters=True

### Conexão com o banco de dados


In [ ]:
conn = duckdb.connect(database=":memory:", read_only=False)

In [ ]:
%sql conn

### Pastas para salvar os arquivos


In [ ]:
# Create data folder if not exists
directories = ["data", "output"]
for directory in directories:
    if not os.path.exists(directory):
        os.makedirs(directory)

### Download dos arquivos


In [ ]:
datasets = [
    {
        "url":"https://public.tableau.com/app/sample-data/sample_-_superstore.xls",
        "file": "data/superstore.xls"
    },
    {
        "url": "https://public.tableau.com/app/sample-data/netflix_titles.xlsx",
        "file": "data/netflix_titles.xlsx"
    },
    {
        "url": "https://public.tableau.com/app/sample-data/titanic%20passenger%20list.csv",
        "file": "data/titanic_passenger_list.csv"
    },
    {
        "url": "https://github.com/plotly/datasets/raw/master/2015_flights.parquet",
        "file": "data/2015_flights.parquet"
    }
]

# Download datasets
for dataset in datasets:
    urlretrieve(dataset["url"], dataset["file"])

### Ler bases de dados


#### Pandas


In [ ]:
# df = DataFrame
# Superstore xls
# https://pandas.pydata.org/docs/reference/api/pandas.read_excel.html
df_superstore_orders = pd.read_excel("data/superstore.xls", sheet_name="Orders")
df_superstore_returns = pd.read_excel("data/superstore.xls", sheet_name="Returns")
df_superstore_people = pd.read_excel("data/superstore.xls", sheet_name="People")
# Netflix xlsx
# https://pandas.pydata.org/docs/reference/api/pandas.read_excel.html
df_netflix_titles = pd.read_excel("data/netflix_titles.xlsx", sheet_name="netflix_titles")
df_netflix_titles_directors = pd.read_excel("data/netflix_titles.xlsx", sheet_name="netflix_titles_directors")
df_netflix_titles_countries = pd.read_excel("data/netflix_titles.xlsx", sheet_name="netflix_titles_countries")
df_netflix_titles_cast = pd.read_excel("data/netflix_titles.xlsx", sheet_name="netflix_titles_cast")
df_netflix_titles_category = pd.read_excel("data/netflix_titles.xlsx", sheet_name="netflix_titles_category")
# Titanic csv
# https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html
df_titanic = pd.read_csv("data/titanic_passenger_list.csv")
# Flights parquet
# https://pandas.pydata.org/docs/reference/api/pandas.read_parquet.html
df_flights = pd.read_parquet("data/2015_flights.parquet")

In [ ]:
df_superstore_orders.head(1)

#### DuckDB


In [ ]:
# https://duckdb.org/docs/guides/python/execute_sql.html
conn.execute(
    """
    INSTALL spatial;
    LOAD spatial;
    """
)

try:
    conn.execute(
        """
        CREATE TABLE IF NOT EXISTS superstore_orders AS
        SELECT *
        FROM st_read('data/superstore.xls', layer='Orders')
        """
    )
except Exception as error:
    print(error)


conn.execute(
    """
    CREATE TABLE IF NOT EXISTS netflix_titles AS
    SELECT *
    FROM st_read('data/netflix_titles.xlsx', layer='netflix_titles')
    """
)

conn.execute("DROP TABLE IF EXISTS netflix_titles")

O DuckDB ainda é muito recente então não possui determinadas funcionalidades (11/2023).

- Especificar o tipo de codificação do arquivo (encoding). Ex: encoding='latin1' (português do Brasil)
- Carregar arquivos .xls

Podemos carregar os dados que se enquadram nestas limitações com o Pandas e depois converter para DuckDB.


In [ ]:
%%sql
CREATE TABLE IF NOT EXISTS superstore_orders AS SELECT * FROM df_superstore_orders;
CREATE TABLE IF NOT EXISTS superstore_returns AS SELECT * FROM df_superstore_returns;
CREATE TABLE IF NOT EXISTS superstore_people AS SELECT * FROM df_superstore_people;

In [ ]:
%%sql
SELECT * FROM superstore_orders LIMIT 1;

In [ ]:
# https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.iterrows.html
def full_text_search(df: pd.DataFrame, search_value: str) -> str:
    """
    Search for a value in all columns of a dataframe
    :param df: Dataframe to search
    :param search_value: Value to search
    :return: String with the search result
    """
    for column in df.columns:
        for index, row in df.iterrows():
            row_value = row[column]
            if isinstance(row_value, str) and search_value in row_value:
                return f'Found "{search_value}" in column "{column}" at index {index}'

    return f'"{search_value}" not found in any column'

In [ ]:
full_text_search(df_netflix_titles, "Flying Fortress")

In [ ]:
# https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.iloc.html
df_netflix_titles.iloc[2018]

In [ ]:
# https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.drop.html
df_netflix_titles.drop(2018, inplace=True)

In [ ]:
%%sql
CREATE TABLE IF NOT EXISTS netflix_titles AS SELECT * FROM df_netflix_titles;
CREATE TABLE IF NOT EXISTS netflix_titles_directors AS SELECT * FROM df_netflix_titles_directors;
CREATE TABLE IF NOT EXISTS netflix_titles_countries AS SELECT * FROM df_netflix_titles_countries;
CREATE TABLE IF NOT EXISTS netflix_titles_cast AS SELECT * FROM df_netflix_titles_cast;
CREATE TABLE IF NOT EXISTS netflix_titles_category AS SELECT * FROM df_netflix_titles_category;


### Operações


#### Pandas


##### Exploração


In [ ]:
# Number of rows
# https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.shape.html
df_superstore_orders.shape[0]

In [ ]:
# Number of columns
# https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.shape.html
df_superstore_orders.shape[1]

In [ ]:
# Statistics
# https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.describe.html
df_superstore_orders.describe()

In [ ]:
# https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.columns.html
df_superstore_orders.columns

In [ ]:
# https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.head.html
df_superstore_orders.head(1)

In [ ]:
# https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.columns.html
df_superstore_orders.columns

In [ ]:
# https://pandas.pydata.org/docs/reference/api/pandas.unique.html
df_superstore_orders["Segment"].unique()

In [ ]:
# https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.nunique.html
df_superstore_orders["Segment"].nunique()

In [ ]:
# https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.head.html
df_superstore_people.head(1)

In [ ]:
# https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.head.html
df_superstore_returns.head(1)

##### Juntar bases


In [ ]:
# https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.merge.html
df_superstore_orders_and_returns = df_superstore_orders.merge(df_superstore_returns, how="left", on="Order ID")
df_superstore_orders_and_returns.head(1)

In [ ]:
# https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.columns.html
df_superstore_orders_and_returns.columns

##### Selecionar colunas especificas


In [ ]:
df_superstore_orders_and_returns[[
    "Order ID",
    "Order Date",
    "Customer Name",
    "Segment",
    "Product ID",
    "Product Name",
    "Quantity",
    "Sales",
    "Profit",
    "Returned"
]].head(1)


##### Filtrar


In [ ]:
# & = and
# | = or
# https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.loc.html
# Rule: Profit > 3000 and Region == South
filter_rule = (
    (df_superstore_orders_and_returns["Profit"] > 3000) &
    (df_superstore_orders_and_returns["Region"] == "South")
)
# Apply rule
df_superstore_orders_and_returns.loc[filter_rule]

##### Condicionais


In [ ]:
# Rule: Profit > 3000 and Region == South
filter_rule = (
    (df_superstore_orders_and_returns["Profit"] > 3000) &
    (df_superstore_orders_and_returns["Region"] == "South")
)
# Apply rule
df_superstore_orders_and_returns.loc[filter_rule, "Returned"] = "Yes"
# Check if rule was applied
df_superstore_orders_and_returns.loc[filter_rule]

##### Agrupar por categoria


In [ ]:
# https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.groupby.html
df_superstore_orders_and_returns_metrics = (
    df_superstore_orders_and_returns.groupby(['Category'])
    .agg({'Profit': 'sum', 'Sales': 'sum', 'Quantity': 'sum', 'Discount': 'sum',})
    .sort_values(by='Profit', ascending=False)
).reset_index()

df_superstore_orders_and_returns_metrics

##### Empilhar tabelas


In [ ]:
_df1 = df_superstore_orders_and_returns_metrics.loc[df_superstore_orders_and_returns_metrics["Category"] == "Technology"]
_df1

In [ ]:
_df2 = df_superstore_orders_and_returns_metrics.loc[df_superstore_orders_and_returns_metrics["Category"] == "Furniture"]
_df2

In [ ]:
# https://pandas.pydata.org/docs/reference/api/pandas.concat.html
df_concat_example = pd.concat([_df1, _df2])
df_concat_example

##### Tabela dinâmica


In [ ]:
# https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.groupby.html
df_superstore_orders_category_region_profit = (
    df_superstore_orders_and_returns.groupby(['Category', "Region"])
    .agg({'Profit': 'sum',})
    .sort_values(by='Profit', ascending=False)
).reset_index()

df_superstore_orders_category_region_profit

In [ ]:
# https://pandas.pydata.org/docs/reference/api/pandas.pivot_table.html
df_superstore_orders_category_region_profit.pivot_table(
    index="Category",
    columns="Region",
    values="Profit",
    aggfunc="sum"
).reset_index()


##### Funções de janela


In [ ]:
# https://pandas.pydata.org/docs/reference/api/pandas.Series.rolling.html
df_superstore_orders_and_returns["month"] = df_superstore_orders_and_returns["Order Date"].dt.month

_df_rolling_window_example = (
    df_superstore_orders_and_returns[["month", "Region", "Profit"]]
    .groupby(["Region", "month"])
    .sum()
    .sort_values(by=["Region", "month"], ascending=True)
    .reset_index()
)

_df_rolling_window_example["cumulated_profit"] = (
    _df_rolling_window_example.groupby(["Region"])["Profit"]
    .rolling(2, min_periods=1)
    .sum()
    .reset_index(drop=True)
)

_df_rolling_window_example.head(20)

##### Exportar dados


In [ ]:
df_superstore_orders_category_region_profit.to_csv(
    "output/superstore_orders_category_region_profit.csv",
    index=False,
    sep=","
)

df_superstore_orders_category_region_profit.to_excel(
    "output/superstore_orders_category_region_profit.xlsx",
    index=False,
)

df_superstore_orders_category_region_profit.to_parquet(
    "output/superstore_orders_category_region_profit.parquet",
    index=False,
)

##### Gráficos


In [ ]:
# https://plotly.com/python/getting-started/
# https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.groupby.html
df_superstore_orders_and_returns_metrics = (
    df_superstore_orders_and_returns.groupby(['Category'])
    .agg({'Profit': 'sum', 'Sales': 'sum', 'Quantity': 'sum', 'Discount': 'sum',})
    .sort_values(by='Profit', ascending=False)
).reset_index()

df_superstore_orders_and_returns_metrics

In [ ]:
# https://pandas.pydata.org/docs/reference/api/pandas.melt.html
df_superstore_orders_and_returns_metrics_chart_data = df_superstore_orders_and_returns_metrics.melt(
    id_vars=["Category"],
    value_vars=["Profit", "Sales", "Quantity", "Discount"],
    var_name="metric",
    value_name="value"
)

df_superstore_orders_and_returns_metrics_chart_data

In [ ]:
fig = px.bar(
    df_superstore_orders_and_returns_metrics_chart_data,
    x="Category",
    y="value",
    color="metric",
    text="value",
    title="Profit by Category",
    hover_data={'value':':.2f'},
    template="seaborn"
)

fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')

fig.update_layout(
    margin=dict(l=20, r=20, t=60, b=20),
)

fig.show()


In [ ]:
# https://plotly.com/python/line-charts/
df_superstore_orders_and_returns["month"] = df_superstore_orders_and_returns["Order Date"].dt.month

_df_superstore_profit_regions_month = (
    df_superstore_orders_and_returns[["month", "Region", "Profit"]]
    .groupby(["Region", "month"])
    .sum()
    .sort_values(by=["Region", "month"], ascending=True)
    .reset_index()
)

fig = px.line(
    _df_superstore_profit_regions_month,
    x="month",
    y="Profit",
    color="Region",
    title="Profit by Region",
    template="seaborn",
    text="Profit",
)

fig.update_traces(texttemplate='%{text:.2s}', textposition='top center')

fig.update_layout(
    margin=dict(l=20, r=20, t=60, b=20),
)

fig.show()

#### DuckDB


##### Exploração


In [ ]:
%%sql
SELECT COUNT(*) FROM superstore_orders;

In [ ]:
%%sql
SELECT COUNT(*)
  FROM INFORMATION_SCHEMA.COLUMNS
 WHERE table_name = 'superstore_orders'

In [ ]:
%%sql
DESCRIBE SELECT * FROM superstore_orders;

In [ ]:
%%sql
SELECT * FROM superstore_orders
LIMIT 1;

In [ ]:
%%sql
SELECT DISTINCT Segment FROM superstore_orders

In [ ]:
%%sql
SELECT COUNT(DISTINCT Segment) AS nunique FROM superstore_orders

In [ ]:
%%sql
SELECT * FROM df_superstore_people
LIMIT 1;

In [ ]:
%%sql
SELECT * FROM df_superstore_returns
LIMIT 1;

##### Juntar bases


In [ ]:
%%sql
DROP TABLE IF EXISTS superstore_orders_and_returns;
CREATE TABLE superstore_orders_and_returns AS
SELECT
    orders.*,
    returns.Returned
FROM df_superstore_orders AS orders
LEFT JOIN df_superstore_returns AS returns
ON orders."Order ID" = returns."Order ID"
LIMIT 1;

SELECT * FROM superstore_orders_and_returns
LIMIT 1;

##### Selecionar colunas especificas


In [ ]:
%%sql
SELECT
    "Order ID",
    "Order Date",
    "Customer Name",
    "Segment",
    "Product ID",
    "Product Name",
    "Quantity",
    "Sales",
    "Profit",
    "Returned"
FROM df_superstore_orders_and_returns
LIMIT 1;

##### Filtrar


In [ ]:
%%sql
SELECT
    "Order ID",
    "Order Date",
    "Customer Name",
    "Segment",
    "Product ID",
    "Product Name",
    "Quantity",
    "Sales",
    "Profit",
    "Returned"
FROM df_superstore_orders_and_returns
WHERE Profit > 3000 AND Region = 'South';

##### Condicionais


In [ ]:
%%sql
SELECT
    "Order ID",
    "Order Date",
    "Customer Name",
    "Segment",
    "Product ID",
    "Product Name",
    "Quantity",
    "Sales",
    "Profit",
    CASE
        WHEN Profit > 3000 AND Region = 'South'THEN 'Yes'
        ELSE Returned
    END AS Returned
FROM df_superstore_orders_and_returns
WHERE Profit > 3000 AND Region = 'South';

##### Agrupar por categoria


In [ ]:
%%sql
DROP TABLE IF EXISTS superstore_orders_and_returns_metrics;
CREATE TABLE superstore_orders_and_returns_metrics AS
SELECT
    Category,
    SUM(Profit) AS Profit,
    SUM(Sales) AS Sales,
    SUM(Quantity) AS Quantity,
    SUM(Discount) AS Discount
FROM df_superstore_orders_and_returns
GROUP BY Category;

SELECT * FROM superstore_orders_and_returns_metrics;

##### Empilhar tabelas


In [ ]:
%%sql
DROP TABLE IF EXISTS tbl_1;
CREATE TABLE tbl_1 AS
SELECT * FROM superstore_orders_and_returns_metrics
WHERE Category = 'Technology';

DROP TABLE IF EXISTS tbl_2;
CREATE TABLE tbl_2 AS
SELECT * FROM superstore_orders_and_returns_metrics
WHERE Category = 'Furniture';


In [ ]:
%%sql
SELECT * FROM tbl_1
UNION
SELECT * FROM tbl_2;

##### Tabela dinâmica


##### Funções de janela


##### Exportar dados


##### CTE (Common Table Expressions)


## Extra

- https://duckdblabs.github.io/db-benchmark/
- Ao lidar com caminhos de arquivos em sistemas operacionais Windows, é necessário utilizar duas barras invertidas (\\) ou uma barra invertida (/) para separar os diretórios. Ex: `C:\\Users\\user\\Documents\\dados\\`
- Para ler arquivos com encoding diferente do padrão, é necessário especificar o encoding. Ex: `pd.read_csv('arquivo.csv', encoding='latin1')`, `pd.read_excel('arquivo.xls', encoding='latin1')`
- Formatos de arquivo como csv e parquet são mais rápidos para ler e escrever do que Excel.
